# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

We encourage you to implement your own versions of the planner that fit your user needs.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [ ]:
!python -m pip install semantic-kernel==0.3.1.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

### Setting Up the Planner
Here we will be using a BasicPlanner that will create a sequential plan to be evaluated in order. The output of a function becomes the inputs of the next function. 

In [ ]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizePlugin` and `WriterPlugin` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [ ]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizePlugin")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterPlugin")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

In [ ]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""
original_plan = await planner.create_plan_async(ask, kernel)

In [ ]:
print(original_plan.generated_plan)

You can see that the Planner took my ask and converted it into an JSON-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan becomes the `input` to the next function. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [ ]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill.

In [ ]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [ ]:
print(new_plan.generated_plan)

### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [ ]:
results = await planner.execute_plan_async(new_plan, kernel)

In [ ]:
print(results)

# Let's see how the planner would work with a ChatCompletion model

Planner works best with more powerful models like `gpt4`, but because that's a ChatCompletion model, we need to modify the kernel to use that instead. Let's see how it works with the cheaper `gpt-35-turbo` model.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion("gpt-35-turbo", endpoint, api_key),
    )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)
    )

In [ ]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizePlugin")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterPlugin")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [ ]:
ask = """
I need to plan a birthday party for my best friend. He loves racecars and only speaks Spanish.
Translate the ideas and make the text in all capital letters."""
chatgpt_plan = await planner.create_plan_async(ask, kernel)

In [ ]:
print(chatgpt_plan.generated_plan)

In [ ]:
results = await planner.execute_plan_async(chatgpt_plan, kernel)

In [ ]:
print(results)